In [2]:
import numpy as np

In [3]:
params = {
    'min_kernel': 2,
    'max_kernel': 9,
    'min_padding': 0,
    'max_padding': 5,
    'min_stride': 1,
    'max_stride': 5
}

In [4]:
def calculate_output_size(input_size, kernel=2, padding=0, stride=1):
    # ((W-F+2*P)/S)+1 must be int
    out = ((input_size - kernel + 2*padding)/stride) + 1
    return out

def find_kernel_stride_padding_options(input_size, output_size,
                                      min_kernel=2, max_kernel=8,
                                      min_padding=0, max_padding=3,
                                      min_stride=1, max_stride=4):
    combs = []
    for k in range(min_kernel,max_kernel+1):
        for p in range(min_padding,max_padding+1):
            for s in range(min_stride,max_stride+1):
                # ((W-F+2*P)/S)+1 must be int
                out = ((input_size - k + 2*p)/s) + 1
                if out == int(out) and out == output_size:
                    combs.append((("k" + str(k),"p" + str(p), "s" + str(s)), out))
    return combs
 
def combos_numpy(n, conditions):
	conditions[0] = 1
	# Create the entire array of possible values
	valid = np.arange(10**(n-1), 10**n, dtype=np.int32)
	# Iterate through the set of conditions
	for digit in conditions:
		# x mod 10^(i + 1) integer divide 10^i will extract the i-th digit of x
		# The equals combined with the sum will tell us the count of the digit in question. This line is basically doing x.count(digit) for every number in the array.
		mask = np.sum(valid % 10**(i+1) // (10**i) == digit for i in range(n))
		# EDIT: It looks like Quora's syntax highlighting views the integer division operator '//' as C style commenting. This is part of the code - not a comment!
		mask = mask < conditions[digit]
		valid = valid[mask]
	return valid

def get_final_output_size(orig_input, kernel=2, stride=2, padding=0, n_layers=1):
    sizes = []
    for _ in range(n_layers):
        out = calculate_output_size(orig_input, kernel=kernel, stride=stride, padding=padding)
        sizes.append(out)
        orig_input = out
    return sizes

#### Original case = 28x28

In [5]:
# Initial layer - 28 --> 32 due to padding=1 ??
get_final_output_size(32, kernel=4, stride=2, padding=1, n_layers=3)

[16.0, 8.0, 4.0]

#### 64x64 case

In [6]:
# In this case initial layer does NOT need to go 64 --> 66 due to padding=1 ???
get_final_output_size(64,kernel=4, stride=2, padding=1, n_layers=4)

[32.0, 16.0, 8.0, 4.0]

#### Tentative 128x128 case

In [7]:
get_final_output_size(128,kernel=4, stride=2, padding=1, n_layers=5)

[64.0, 32.0, 16.0, 8.0, 4.0]

In [8]:
128*2

256

In [9]:
calculate_output_size(260, kernel=4, stride=2, padding=1)

130.0

In [29]:
find_kernel_stride_padding_options(224,117,**params)

[(('k2', 'p5', 's2'), 117.0)]

In [40]:
z = []
z.append(2)
z

[2]

In [66]:
def find_two_layer_options(reduced_dim,starting_dim):
    output = []
    options = []
    for i in range(reduced_dim+1,starting_dim-1):
        x = find_kernel_stride_padding_options(starting_dim,i,**params)
        if x:
            output.append(i)

    for option in output:
        y = find_kernel_stride_padding_options(option,reduced_dim,**params)
        if y:
            options.append(y)
            
    return options

In [72]:
results = []
for i in range(198,250):
    x = find_two_layer_options(128,i)
    if x:
        print(i,x)
        results.append((i,x))

248 [[(('k2', 'p0', 's1'), 128.0), (('k4', 'p1', 's1'), 128.0), (('k6', 'p2', 's1'), 128.0), (('k8', 'p3', 's1'), 128.0)], [(('k2', 'p5', 's2'), 128.0)]]
249 [[(('k2', 'p0', 's1'), 128.0), (('k4', 'p1', 's1'), 128.0), (('k6', 'p2', 's1'), 128.0), (('k8', 'p3', 's1'), 128.0)], [(('k2', 'p5', 's2'), 128.0)], [(('k3', 'p5', 's2'), 128.0)]]


In [77]:
calculate_output_size(249,kernel=4,padding=1,stride=1)

248.0

In [78]:
128+128

256

In [79]:
get_final_output_size(256,kernel=4, stride=2, padding=1, n_layers=6)

[128.0, 64.0, 32.0, 16.0, 8.0, 4.0]

In [14]:
find_kernel_stride_padding_options(128,32,**params)

[(('k4', 'p0', 's4'), 32.0),
 (('k6', 'p1', 's4'), 32.0),
 (('k8', 'p2', 's4'), 32.0)]

In [16]:
def find_first_n_results(in_size, n):
    all_results = []
    num_results = 0
    orig_in_size = in_size
    while in_size >= 4:
        in_size -= 2
        results = find_kernel_stride_padding_options(orig_in_size, in_size, **params)
        if results:
            num_results +=1
            all_results.extend(results)
        if num_results >= n:
            return all_results
    return all_results

In [21]:
sorted(find_first_n_results(128,30))

[(('k2', 'p1', 's2'), 66.0),
 (('k2', 'p2', 's4'), 34.0),
 (('k2', 'p3', 's2'), 68.0),
 (('k3', 'p0', 's1'), 128.0),
 (('k3', 'p1', 's3'), 44.0),
 (('k3', 'p4', 's3'), 46.0),
 (('k3', 'p4', 's5'), 28.0),
 (('k4', 'p0', 's2'), 64.0),
 (('k4', 'p2', 's2'), 66.0),
 (('k4', 'p3', 's4'), 34.0),
 (('k4', 'p4', 's2'), 68.0),
 (('k5', 'p0', 's1'), 126.0),
 (('k5', 'p0', 's5'), 26.0),
 (('k5', 'p1', 's1'), 128.0),
 (('k5', 'p2', 's3'), 44.0),
 (('k6', 'p0', 's4'), 32.0),
 (('k6', 'p1', 's2'), 64.0),
 (('k6', 'p3', 's2'), 66.0),
 (('k6', 'p4', 's4'), 34.0),
 (('k7', 'p0', 's1'), 124.0),
 (('k7', 'p0', 's3'), 42.0),
 (('k7', 'p1', 's1'), 126.0),
 (('k7', 'p1', 's5'), 26.0),
 (('k7', 'p2', 's1'), 128.0),
 (('k7', 'p3', 's3'), 44.0),
 (('k8', 'p0', 's2'), 62.0),
 (('k8', 'p1', 's4'), 32.0),
 (('k8', 'p2', 's2'), 64.0),
 (('k8', 'p4', 's2'), 66.0),
 (('k9', 'p0', 's1'), 122.0),
 (('k9', 'p1', 's1'), 124.0),
 (('k9', 'p1', 's3'), 42.0),
 (('k9', 'p2', 's1'), 126.0),
 (('k9', 'p2', 's5'), 26.0),
 (('k

In [73]:
find_kernel_stride_padding_options(260,128,**params)

[(('k6', 'p0', 's2'), 128.0), (('k8', 'p1', 's2'), 128.0)]

In [74]:
find_kernel_stride_padding_options(128,64,**params)

[(('k2', 'p0', 's2'), 64.0),
 (('k4', 'p1', 's2'), 64.0),
 (('k6', 'p2', 's2'), 64.0),
 (('k8', 'p3', 's2'), 64.0)]

In [72]:
find_kernel_stride_padding_options(64,32,**params)

[(('k2', 'p0', 's2'), 32.0),
 (('k4', 'p1', 's2'), 32.0),
 (('k6', 'p2', 's2'), 32.0),
 (('k8', 'p3', 's2'), 32.0)]

In [71]:
find_kernel_stride_padding_options(256,254,**params)

[(('k3', 'p0', 's1'), 254.0),
 (('k5', 'p1', 's1'), 254.0),
 (('k7', 'p2', 's1'), 254.0),
 (('k9', 'p3', 's1'), 254.0)]

In [13]:
from itertools import combinations
comb = combinations([1,2,34],2)
list(comb)

[(1, 2), (1, 34), (2, 34)]

In [8]:
combos_numpy(3,[1,2,3])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  # This is added back by InteractiveShellApp.init_path()


IndexError: list index out of range

#### `torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)`

#### Applies a 2D transposed convolution operator over an input image composed of several input planes.

#### This module can be seen as the gradient of Conv2d with respect to its input.

* It is also known as a fractionally-strided convolution or a deconvolution (although it is not an actual deconvolution operation).

In [75]:
import torch

In [76]:
torch.nn.ConvTranspose2d()

AttributeError: module 'torch.nn' has no attribute 'ConvTranspose2D'